### Loading our vectore database

In [1]:
from vector_db_manager import VectoreDatabaseManager

vdb_manager = VectoreDatabaseManager()
vector_db = vdb_manager.get_vector_databaase()

/Users/mohamedhassan/Projects/VoiceBot_with_RAG/vector_db_manager.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
/Users/mohamedhassan/miniconda3/envs/llms/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Creating new vector database ...
Processing: data/knowledge_base/الاسئلة والأجوبة.txt
Processing: data/knowledge_base/مركز_وقاء.txt
Processing: data/knowledge_base/future_plateform.txt
Processing: data/knowledge_base/بيانات المؤتمر.txt
Processing: data/knowledge_base/original_fiels/Mohamed Hassan.pdf
Processing: data/knowledge_base/original_fiels/الاسئلة و الجوبة.pdf


In [6]:
all_documents = vector_db.docstore._dict.values()

In [16]:
all_chunks = []
for i, doc in enumerate(all_documents):
    all_chunks.append({
        "chunk_id": doc.metadata['chunk_id'],
        "source_file_path": doc.metadata['file_path'],
        "chunk_content": doc.page_content
    })


In [17]:
import pandas as pd

df = pd.DataFrame(all_chunks)
df.head()

,chunk_id,source_file_path,chunk_content
0,الاسئلة والأجوبة_chunk_0,data/knowledge_base/الاسئلة والأجوبة.txt,سؤال 1: ماهي الزراعة العضوية؟\n\n\nجواب 1:\nال...
1,الاسئلة والأجوبة_chunk_1,data/knowledge_base/الاسئلة والأجوبة.txt,سؤال 2: ماهي مبادئ الزراعة العضوية؟\n\nجواب 2:...
2,الاسئلة والأجوبة_chunk_2,data/knowledge_base/الاسئلة والأجوبة.txt,البيئة:\nتقوم الزراعة العضوية على احترام الأنظ...
3,الاسئلة والأجوبة_chunk_3,data/knowledge_base/الاسئلة والأجوبة.txt,العدالة:\nتلتزم الزراعة العضوية بالعلاقات العا...
4,الاسئلة والأجوبة_chunk_4,data/knowledge_base/الاسئلة والأجوبة.txt,سؤال 3: ماهي فوائد الزراعة العضوية؟\n\nجواب 3:...


In [36]:
topics = df.source_file_path.unique()
topics

array(['data/knowledge_base/الاسئلة والأجوبة.txt',
       'data/knowledge_base/مركز_وقاء.txt',
       'data/knowledge_base/future_plateform.txt',
       'data/knowledge_base/بيانات المؤتمر.txt',
       'data/knowledge_base/original_fiels/Mohamed Hassan.pdf',
       'data/knowledge_base/original_fiels/الاسئلة و الجوبة.pdf'],
      dtype=object)

### Now, we could make our Evaluation data for our RAG System

In [43]:
def get_me_chunk_of_topic(file_path):
    """
    Function that help us print all chunks of each topic so that we could make some questions and hummanly (or with LLMs) select relevant questions
    """
    focused_df = df.query(f"source_file_path == '{file_path}'")
    for i in range(focused_df.shape[0]):
        print(f'============= {focused_df.iloc[i, 0]} =============')
        print(focused_df.iloc[i, 2])
        print()

#### 1) data/knowledge_base/الاسئلة والأجوبة.txt

In [44]:
get_me_chunk_of_topic('data/knowledge_base/الاسئلة والأجوبة.txt')

============= الاسئلة والأجوبة_chunk_0 =============
سؤال 1: ماهي الزراعة العضوية؟


جواب 1:
الزراعة العضوية هي أسلوب إنتاج زراعي مستدام يشمل الزراعة النباتية والحيوانية، ويعتمد على استخدام مواد طبيعية لإنتاج غذاء صحي وآمن دون اللجوء إلى مواد كيميائية أو أسمدة أو مبيدات، كما يتم تجنب استخدام هرمونات أو كائنات معدلة وراثياً أو مواد مشتقة.

سؤال 2: ماهي مبادئ الزراعة العضوية؟

جواب 2:
تعتمد الزراعة العضوية على أربع مبادئ أساسية:

============= الاسئلة والأجوبة_chunk_1 =============
سؤال 2: ماهي مبادئ الزراعة العضوية؟

جواب 2:
تعتمد الزراعة العضوية على أربع مبادئ أساسية:

الصحة:
تساهم الزراعة العضوية في الحفاظ على صحة الإنسان، النبات، والحيوان، وتعتبر الأرض وحدة لا تتجزأ؛ حيث ترتبط صحة الأفراد والمجتمعات بصحة الأنظمة البيئية. التربة الصحية تنتج محاصيل صحية تغذي الإنسان والحيوان.

============= الاسئلة والأجوبة_chunk_2 =============
البيئة:
تقوم الزراعة العضوية على احترام الأنظمة البيئية الحية والدورات الطبيعية، حيث تعمل على استدامتها. يجب أن يعتمد الإنتاج العضوي على الأساليب البيئية وإعاد

In [49]:
organic_farming_eval_set = [
    {
        "question": "ما هي الزراعة العضوية؟",
        "relevant_chunks": ["الأسئلة والأجوبة_chunk_0"]
    },
    {
        "question": "ما هي مبادئ الزراعة العضوية؟",
        "relevant_chunks": ["الأسئلة والأجوبة_chunk_1", "الأسئلة والأجوبة_chunk_2", "الأسئلة والأجوبة_chunk_3"]
    },
    {
        "question": "ما هي فوائد الزراعة العضوية؟",
        "relevant_chunks": ["الأسئلة والأجوبة_chunk_4", "الأسئلة والأجوبة_chunk_5", "الأسئلة والأجوبة_chunk_6"]
    },
    {
        "question": "كيف أتعرف على المنتجات العضوية؟",
        "relevant_chunks": ["الأسئلة والأجوبة_chunk_8"]
    },
    {
        "question": "هل يوجد دعم حكومي للإنتاج العضوي في المملكة؟",
        "relevant_chunks": ["الأسئلة والأجوبة_chunk_13"]
    }
]

#### 2) data/knowledge_base/مركز_وقاء.txt

In [45]:
get_me_chunk_of_topic('data/knowledge_base/مركز_وقاء.txt')

============= مركز_وقاء_chunk_0 =============
مركز وقاء هو المركز الوطني المتخصص في الوقاية من الآفات النباتية والأمراض الحيوانية في المملكة العربية السعودية، حيث يعمل على تنفيذ سياسات وزارة البيئة والمياه والزراعة لضمان صحة واستدامة النباتات والحيوانات. يهدف إلى تعزيز مشاركة القطاع الخاص وتوحيد الجهود بين الجهات الحكومية والخاصة في هذا المجال.

============= مركز_وقاء_chunk_1 =============
رادار وقاء هو نظام يراقب البؤر الحيوانية لحظياً عبر خرائط رقمية، حيث يحدد مستوى الخطورة لكل بؤرة، ويمكن من خلاله الوصول إلى بيانات المربين والمسالخ والمشاريع الزراعية القريبة من المناطق المصابة، مما يساهم في حماية الثروة الحيوانية عبر الاستجابة السريعة للطوارئ وتحليل المخاطر المحيطة.

============= مركز_وقاء_chunk_2 =============
نظام مهامي هو أداة لإدارة وتتبع المهام داخل المركز، حيث يساعد في تنظيم العمل، تحديد الأولويات، تعزيز التعاون بين الفرق، وتقدير الوقت والموارد المطلوبة، مع إمكانية متابعة التقدم في إنجاز المهام واستخراج التقارير اللازمة لضمان الالتزام بالجداول الزمنية.

============= مركز_وق

In [50]:
weqaa_center_eval_set = [
    {
        "question": "ما هو مركز وقاء وما أهدافه؟",
        "relevant_chunks": ["مركز_وقاء_chunk_0"]
    },
    {
        "question": "ما هو نظام رادار وقاء وما أهميته؟",
        "relevant_chunks": ["مركز_وقاء_chunk_1"]
    },
    {
        "question": "ما هي الخدمات التي توفرها منصة خدمات مستفيدين الأعمال؟",
        "relevant_chunks": ["مركز_وقاء_chunk_6"]
    },
    {
        "question": "ما هو نظام إدارة الاستراتيجية وقياس الأداء؟",
        "relevant_chunks": ["مركز_وقاء_chunk_3"]
    },
    {
        "question": "ما هو برنامج “رفاه” وما أهدافه؟",
        "relevant_chunks": ["مركز_وقاء_chunk_7"]
    }
]